**Какие библиотеки можно использовать?**

nltk, TextBlo, Scikit–learn и другие.

Приведение к нижнему регистру -> очистка/обработка спец.символов -> удаление стоп-слов -> лемматизация/стемминг -> токенизация(нормализация)

Токенизация (иногда – сегментация) по словам – это процесс разделения предложений на слова-компоненты.

**Гипотезы бизнес-правил**

1. Если мы избавимся от селлеров озона, использующих площадку в своих интересах (не для продажи, а для рекламы), то мы получим больше потенциальных клиентов, которые купят товар на Озоне и принесут прибыль в виде комиссии.
2. Для того, чтобы это проверить, мы воспользуемся нейросетями для решения данной проблемы.
3. Мы измерим конверсию ДО введения нейросети в продакшен и ПОСЛЕ и сравним результаты.

In [290]:
%matplotlib inline
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid') # стиль графиков 

In [69]:
dataset = pd.read_csv(r'C:\Users\dmitr\Уник\Проф\train\train.csv')
dataset.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0


In [85]:
ds_is_bad = dataset[dataset['is_bad'] == 1]

In [86]:
ds_is_bad

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1
5,Зимние шины Hankook Winter i*Pike RS6 W569,Размеры шин Hankook Winter i*Pike RS1 W319. Пр...,Запчасти и аксессуары,Транспорт,11000.0,Московская область,Железнодорожный,2019-06-01 00:01:03.471366,1
6,"LADA Priora, 5046",приора 918 норма+кондинционер. 014 машина 16 в...,Автомобили,Транспорт,340000.0,Чеченская Республика,Грозный,2019-06-01 00:01:13.603386,1
10,"Окрашивание элемента 1000,автосервис,сто","АВТОСЕРВИС ""ВИКИНГ""/\n-ОКРАШИВАНИЕ ЭЛЕМЕНТА от...",Предложение услуг,Услуги,1000.0,Россия,Санкт-Петербург,2019-06-01 00:02:35.822686,1
...,...,...,...,...,...,...,...,...,...
984461,"Детская обувь,осенне-зимняя","Детская обувь, белые зимние сказка размер 46 с...",Детская одежда и обувь,Личные вещи,500.0,Краснодарский край,Кропоткин,2019-10-09 23:43:27.314313,1
984477,Трактор Т-40,Все вопросы по телефону :89029780487 Алексей,Грузовики и спецтехника,Транспорт,50000.0,Смоленская область,Гагарин,2019-10-09 23:54:01.019942,1
984480,"6-к квартира, 45.2 м², 7/9 эт.",Продаю 3-ух комнатную квартиру в кирпичном 9 э...,Квартиры,Недвижимость,1800000.0,Калужская область,Калуга,2019-10-09 23:55:41.822449,1
984482,Штампы 61r,+79780979761/\nДенис/\nширину обода и вылет на...,Запчасти и аксессуары,Транспорт,2000.0,Крым,Мирное,2019-10-09 23:55:54.099019,1


In [71]:
dataset[dataset['is_bad'] == 0].head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0
7,Дверь входная,"Продам дверь входную, данная дверь стояла от з...",Ремонт и строительство,Для дома и дачи,3000.0,Россия,Санкт-Петербург,2019-06-01 00:01:33.818452,0
8,Джинсы фирмы Gulliver,Продаю джинсы фирмы Gulliver. Отличная посадка.,Детская одежда и обувь,Личные вещи,500.0,Россия,Москва,2019-06-01 00:01:41.136540,0


In [72]:
ds_redacted = dataset[['title', 'description', 'is_bad']]
ds_redacted['description']

0         Продаем диван-кровать. Удобный механизм - евро...
1         Кожух рулевой колонки DAF XF 94 (60066004)/\n ...
2         ! Буст аккаунтов с ммр выше 1000ммр не беру ! ...
3         Продам телевизор . Диагональ 450.наличие входа...
4         Открытки-конверты ручной работы/\nВыполнены в ...
                                ...                        
984482    +79780979761/\nДенис/\nширину обода и вылет на...
984483                           Зарядка и под в комплекте.
984484    Белый стул Ikea /\nСостояние требует ремонта: ...
984485    Бронирование с октября по январь 1010 г./\n   ...
984486    Основное и самое любимое направление моей рабо...
Name: description, Length: 984487, dtype: object

In [73]:
dp = dataset[['region', 'is_bad']]

In [74]:
dp['region'].value_counts()

Россия                  237785
Краснодарский край       52960
Московская область       50261
Татарстан                33361
Свердловская область     27943
                         ...  
Магаданская область        503
Тыва                       190
Сахалинская область        168
Ненецкий АО                 60
Чукотский АО                27
Name: region, Length: 85, dtype: int64

In [75]:
dp = dp.groupby('region').count()

In [76]:
dp = dp.sort_values(by=['is_bad'], ascending=False)

In [80]:
pprint(dp.to_string())

('                         is_bad\n'
 'region                         \n'
 'Россия                   237785\n'
 'Краснодарский край        52960\n'
 'Московская область        50261\n'
 'Татарстан                 33361\n'
 'Свердловская область      27943\n'
 'Ростовская область        26725\n'
 'Башкортостан              23956\n'
 'Нижегородская область     23868\n'
 'Челябинская область       23610\n'
 'Самарская область         23478\n'
 'Новосибирская область     20429\n'
 'Пермский край             17061\n'
 'Крым                      16702\n'
 'Ставропольский край       15029\n'
 'Воронежская область       14716\n'
 'Красноярский край         13827\n'
 'Дагестан                  13577\n'
 'Волгоградская область     13223\n'
 'Кировская область         13148\n'
 'Омская область            12959\n'
 'Саратовская область       12390\n'
 'Кемеровская область       11920\n'
 'Алтайский край            11820\n'
 'Иркутская область         11369\n'
 'Тюменская область         10797\n'
 

In [103]:
pprint(ds_is_bad[:10]['description'].values.tolist())

['! Буст аккаунтов с ммр выше 1000ммр не беру ! /\n'
 'Отвечу сразу на вопрос: почему здесь?/\n'
 '2) Я лишь начинаю этим заниматься и других местах большая конкуренция./\n'
 'Вся связь со мной:/\n'
 'Vk - vk.com/id153740153/\n'
 'Discord - Goku#4407',
 'Продам телевизор . Диагональ 450.наличие входа USB для подключения '
 'накопителей и специализированных многофункциональных устройств;/\n'
 '/\n'
 'функция приема сигнала сети Интернет;/\n'
 '/\n'
 'возможность останавливать трансляцию ТВ-программ при помощи Time Shift;/\n'
 '/\n'
 'технология Quattron, улучшающая цветопередачу экрана;/\n'
 '/\n'
 'сканирующая подсветка, не допускающая смазывания кадров;/\n'
 '/\n'
 'имеющаяся  функция Skype Ready;/\n'
 '/\n'
 'комплектование жидкокристаллическими панелями последнего поколения X-gen;/\n'
 '/\n'
 'система Aquos.',
 'Размеры шин Hankook Winter i*Pike RS1 W319. Пробег шин 5000 км. Все шипы на '
 'месте. За комплект 44000 руб. Номер телефона +7 945 679-60-59',
 'приора 918 норма+кондинцион

In [201]:
pprint(ds_is_bad[31:60]['description'].values.tolist())

['Есть возможность приобретения квартиры на территории России, Казахстана, '
 'Киргизии и Белоруссии в РАССРОЧКУ.Заиниересовало?Запишись на консультацию '
 '89493636061',
 'Продуется колеса с дисками штампованными , плюс колпаки , состоянии идеально '
 'износ не более 20 процентов ‘снял с ладе Веста .22 тысяч торг.звонить сюда '
 '89159567870 размер р 25 летние',
 'Продам форд, на полном ходу, пробег 680000 км. Есть недочёты на бамперах.  '
 '893382 торг 11642',
 'Есть кровельные Опоры около 20шт. Без нижней резини',
 'Звоните: +7 911 862 42 22. Сергей.',
 'Продаётся благоустроенная квартира со всеми коммуникациями в пгт. Пристень, '
 'по ул. Кирова /\n'
 'Обращаться по номеру тел: 8-960-689-05-70',
 'Продаю студию в черновой отделке на 35 этаже 37 этажного панельного дома по '
 'адресу ул.Энтузиастов 35 к 3.Сделана стяжка,электроразводка,установлены '
 'стояки горячей и холодной воды и водосчетчики,металлическая входная '
 'дверь.Окна выходят на новый детский сад и новую школу.Цена 90

**Обратная связь**

+7 945 679-60-59\
vk.com/id153740153\
Discord, +7 945 679-60-59\
8936991941(Звонок,What's App)\
89 57 082 43 85\
Вацсап 8в9в1в8в6в4в8в9в4в1в4в\
не звонит\
989звон352по79этому32ном(вотсап)\
вацап восемь девятсот восемнадцать четыре три\
девятки четырестапять\
8-905-792-44-85\
89408288167\
016 наб 8 дивитсот двацат восем 490 нол пят\
вацап восемь девятсот\
восемнадцать четыре три девятки четырестапять\
89O1571O8O3\
893382 торг 11642\
8 9640395609\
8 9 2 8 4 9 4 8 2 2 7\
8989@849@80@81@\
артикул 9010508272\
ВАЦАП восемь девятьсот девять четыреста двенадцать шестьдесят пять восемьдесят\


In [207]:
dataset.groupby('city').count().sort_values(by=['is_bad'], ascending=False)

,title,description,subcategory,category,price,region,datetime_submitted,is_bad
city,,,,,,,,
Москва,158508,158508,158508,158508,150661,158508,158508,158508
Санкт-Петербург,79277,79277,79277,79277,76770,79277,79277,79277
Краснодар,23619,23619,23619,23619,22428,23619,23619,23619
Казань,20161,20161,20161,20161,19052,20161,20161,20161
Екатеринбург,19132,19132,19132,19132,18467,19132,19132,19132
...,...,...,...,...,...,...,...,...
Кондинское,1,1,1,1,1,1,1,1
Темиртау,1,1,1,1,1,1,1,1
Новописцово,1,1,1,1,1,1,1,1


In [210]:
ds_is_bad_red = ds_is_bad[['title', 'description', 'subcategory', 'category']]

In [211]:
ds_is_bad_red

,title,description,subcategory,category
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника
5,Зимние шины Hankook Winter i*Pike RS6 W569,Размеры шин Hankook Winter i*Pike RS1 W319. Пр...,Запчасти и аксессуары,Транспорт
6,"LADA Priora, 5046",приора 918 норма+кондинционер. 014 машина 16 в...,Автомобили,Транспорт
10,"Окрашивание элемента 1000,автосервис,сто","АВТОСЕРВИС ""ВИКИНГ""/\n-ОКРАШИВАНИЕ ЭЛЕМЕНТА от...",Предложение услуг,Услуги
...,...,...,...,...
984461,"Детская обувь,осенне-зимняя","Детская обувь, белые зимние сказка размер 46 с...",Детская одежда и обувь,Личные вещи
984477,Трактор Т-40,Все вопросы по телефону :89029780487 Алексей,Грузовики и спецтехника,Транспорт
984480,"6-к квартира, 45.2 м², 7/9 эт.",Продаю 3-ух комнатную квартиру в кирпичном 9 э...,Квартиры,Недвижимость
984482,Штампы 61r,+79780979761/\nДенис/\nширину обода и вылет на...,Запчасти и аксессуары,Транспорт


In [218]:
ds_is_bad_red.groupby('category').count()['title'].sort_values(ascending=False)

category
Транспорт              54813
Недвижимость           36694
Для дома и дачи        30087
Услуги                 28430
Личные вещи            27509
Бытовая электроника    21010
Животные               17159
Хобби и отдых          10475
Работа                 10340
Для бизнеса             1749
Name: title, dtype: int64

In [223]:
ds_is_bad_red['title'].iloc[:30]

2                          Дешёвый буст аккаунтов Dota 4
3                      Телевизор sharp.Смарт тв.Интернет
5             Зимние шины Hankook Winter i*Pike RS6 W569
6                                      LADA Priora, 5046
10              Окрашивание элемента 1000,автосервис,сто
12                                     Ворота с калиткой
14                                Радиатор на Деу Нексия
16                          5-к квартира, 47 м², 5/2 эт.
17                                     LADA Priora, 4051
21                                                Корова
38     Строительная бригада предлагаем свои услуги ст...
41                          Камера Sаmsung S40+.Оригинал
42                                    Молокоотсос Chicco
49                   Замена радиаторов печки калина-нива
51                                           Прицеп Кунг
53                                     LADA Priora, 4023
54                                     Продам тнвд с смд
56                             

In [237]:
pst = PorterStemmer()

In [267]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dmitr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [272]:
text = ''.join(map(str, ds_is_bad[:3000]['description'].values.tolist()))

In [283]:
sents = sent_tokenize(text)

In [284]:
words = word_tokenize(text)

In [285]:
text_nltk = nltk.Text(words)

In [286]:
fdist = FreqDist(text_nltk)

In [299]:
fdist

FreqDist({',': 17360, '/': 13650, '.': 8592, '↓': 7008, 'и': 6238, '!': 5639, 'в': 4774, ')': 4771, '(': 4578, 'на': 4141, ...})

In [292]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dmitr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [293]:
russian_stopwords = stopwords.words("russian")

In [307]:
fdist_sw = FreqDist(text)
russian_stopwords.extend([';', '/', '!', '?', '(', ')', ',', '.', '↓'])
filtered_words = [word for word in words if word not in russian_stopwords]

In [308]:
text_nltk = nltk.Text(filtered_words)
fdist = FreqDist(text_nltk)
fdist

FreqDist({':': 3821, '-': 1791, 'этаж': 1699, '✔': 1680, 'ТЦ': 1648, '—': 1444, "''": 1160, '``': 1119, 'работы': 972, '0': 868, ...})